In [ ]:
# ===================================================================
# CELDA 1: PREPARACIÓN DE DATOS (MISE EN PLACE)
# ===================================================================

# --- Dependencias principales ---
import numpy
import surprise
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
import os
import mlflow
import mlflow.sklearn
from pathlib import Path

# --- Información de entorno ---
print("--- Fase de Preparación ---")
print(f"Numpy version: {numpy.__version__}")
print(f"Surprise version: {surprise.__version__}")

# --- Resolución de la raíz del proyecto ---
try:
    BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
except NameError:
    if "notebooks" in os.getcwd():
        BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
    else:
        BASE_DIR = os.getcwd()
print(f"Raíz del proyecto detectada en: {BASE_DIR}")

# --- Configuración universal de MLflow ---
tracking_uri = "http://localhost:5000"
mlflow.set_tracking_uri(tracking_uri)
print(f"MLflow client configured to communicate with server at: {tracking_uri}")
mlflow.set_experiment("LatentLens-SVD-Evaluation")

# --- Carga y muestreo de datos MovieLens ---
print("Cargando y procesando datos... (Esto puede tardar un poco)")
ratings_path = Path(BASE_DIR) / 'data' / 'ml-25m' / 'ratings.csv'
ratings_df = pd.read_csv(ratings_path)

n_users = 40000
n_movies = 20000
user_ids = ratings_df['userId'].value_counts().nlargest(n_users).index
movie_ids = ratings_df['movieId'].value_counts().nlargest(n_movies).index
sampled_df = ratings_df[(ratings_df['userId'].isin(user_ids)) & (ratings_df['movieId'].isin(movie_ids))]

# --- Conversión a formato Surprise y división ---
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(sampled_df[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

print("\n¡Mise en Place completado! Las variables 'trainset' y 'testset' están listas en memoria.")

--- Fase de Preparación ---
Numpy version: 1.26.4
Surprise version: 1.1.4
Raíz del proyecto detectada en: c:\Users\Gat\Documents\GitHub\LatentLens
MLflow configurado para comunicarse con el servidor en: http://localhost:5000
MLflow configurado para guardar en: http://localhost:5000


In [ ]:
# ===================================================================
# CELDA 2: EXPERIMENTO Y ENTRENAMIENTO (VERSIÓN FINAL DE VERDAD)
# ===================================================================
import pickle
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec
import pandas as pd
import numpy as np # Importamos numpy para el control de tipos

class SurpriseWrapper(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        with open(context.artifacts["surprise_model_path"], 'rb') as f:
            self.model = pickle.load(f)

    def predict(self, context, model_input: pd.DataFrame) -> pd.Series:
        users = model_input["userId"]
        items = model_input["movieId"]
        predictions = [self.model.predict(uid=user, iid=item).est for user, item in zip(users, items)]
        return pd.Series(predictions)

run_name = "SVD_PRODUCTION_READY_FINAL_V2"
print(f"--- Iniciando experimento: '{run_name}' ---")

with mlflow.start_run(run_name=run_name) as run:
    # ... (parámetros y entrenamiento sin cambios) ...
    n_factors = 150
    n_epochs = 20
    mlflow.log_param("model_type", "SVD")
    mlflow.log_param("n_factors", n_factors)
    mlflow.log_param("n_epochs", n_epochs)
    
    model = SVD(n_factors=n_factors, n_epochs=n_epochs, random_state=42)
    model.fit(trainset)
    
    predictions = model.test(testset)
    rmse = accuracy.rmse(predictions)
    mlflow.log_metric("rmse", rmse)
    
    # El Blueprint explícito y perfecto.
    input_schema = Schema([
        ColSpec("integer", "userId"),
        ColSpec("integer", "movieId"),
    ])
    output_schema = Schema([ColSpec("double")])
    signature = ModelSignature(inputs=input_schema, outputs=output_schema)
    
    # ¡¡¡LA CORRECCIÓN DEFINITIVA ESTÁ AQUÍ!!!
    # Le decimos a Pandas que cree las columnas con el tipo de dato
    # de numpy `np.int32`, que es el que MLflow espera para un `integer`.
    input_example = pd.DataFrame({
        "userId": pd.Series([1], dtype=np.int32),
        "movieId": pd.Series([10], dtype=np.int32)
    })
    
    model_path = "svd_model.pkl"
    with open(model_path, 'wb') as f:
        pickle.dump(model, f)
        
    mlflow.pyfunc.log_model(
    # La Llave Maestra: Usamos el nombre de argumento moderno y correcto.
    name="surprise_svd_model",

    # El Manual de Instrucciones: Le damos nuestro Wrapper personalizado.
    python_model=SurpriseWrapper(),

    # La Caja de Herramientas: Le decimos qué archivos necesita el Wrapper.
    artifacts={"surprise_model_path": model_path},

    # El Ejemplo Perfecto: Un ejemplo de entrada con los tipos de datos EXACTOS.
    input_example=input_example,

    # El Contrato Inmutable: El blueprint explícito que evita toda ambigüedad.
    signature=signature)
    print("Modelo SVD (artefacto) y métricas registradas con firma de producción final.")

print(f"\n¡Experimento '{run_name}' finalizado!")

--- Iniciando experimento: 'SVD_PRODUCTION_READY_FINAL_V2' ---
RMSE: 0.7433


2025/08/12 13:34:54 INFO mlflow.pyfunc: Validating input example against model signature


Modelo SVD (artefacto) y métricas registradas con firma de producción final.

¡Experimento 'SVD_PRODUCTION_READY_FINAL_V2' finalizado!


: 